In [1]:
import pickle
import os
import util
import set_expan
import time
import os
import json
import random
import copy
import sys

In [2]:
def ap(lst, truth):
    rel = 0
    ap = 0
    for idx in range(1, len(lst)+1):
        if lst[idx-1] in truth:
            rel += 1
            ap += rel/idx
    return ap/len(truth)

In [8]:
FLAGS_USE_TYPE=True
cur_dir = os.path.join(os.getcwd())
data = "wiki"
print('dataset:%s' % data)
folder = '/../../data/'+data+'/intermediate/'
start = time.time()
print('loading eid and name maps')
eid2ename, ename2eid = util.loadEidToEntityMap(cur_dir + folder+'entity2id.txt') #entity2id.txt
print('loading eid and skipgram maps')
eid2patterns, pattern2eids = util.loadFeaturesAndEidMap(cur_dir + folder+'reduced_eidSkipgramCounts.txt') #eidSkipgramCount.txt
print('loading skipgram strength map')
eidAndPattern2strength = util.loadWeightByEidAndFeatureMap(cur_dir + folder+'setexpan_eidSkipgram2TFIDFStrength.txt', idx=-1) #(eid, feature, weight) file
print('loading eid and type maps')
eid2types, type2eids = util.loadFeaturesAndEidMap(cur_dir + folder+'eidTypeCounts.txt') #eidTypeCount.txt
print('loading type strength map')
eidAndType2strength = util.loadWeightByEidAndFeatureMap(cur_dir + folder+'eidType2TFIDFStrength.txt', idx=-1) #(eid, feature, weight) file
end = time.time()
print("Finish loading all dataset, using %s seconds" % (end-start))

dataset:wiki
loading eid and name maps
loading eid and skipgram maps
loading skipgram strength map
loading eid and type maps
loading type strength map
Finish loading all dataset, using 52.53902721405029 seconds


In [4]:
good_gold_set = {}
for filename in os.listdir('../../data/eval/cleaned_set/'):
    with open('../../data/eval/cleaned_set/'+filename, 'r') as fin:
        setname = filename.split('.')[0]
        data = fin.readlines()
        ents = []
        for line in data:
            ents.append(line.strip('\n'))
        eids = [ename2eid[x] for x in ents]
        good_gold_set[setname] = eids

In [12]:
## Start set expansion
for st in good_gold_set:
    entities = good_gold_set[st]
    ap2 = []
    for numseeds in range(2,4):
        start = time.time()
        ap1 = []
        for idx in range(1, 3):
            print("running {} [{} seeds] iteration {}".format(st, numseeds, idx))
            samples = random.sample(entities, numseeds)
            entities_to_retrieve = [i for i in entities if i not in samples]
            truth = [eid2ename[int(x)] for x in entities_to_retrieve] # names
            seedEidsWithConfidence = [(int(ele), 0.0) for ele in samples]
            print("input: {}\n".format(str([eid2ename[int(x)] for x in samples])))

            negativeSeedEids = set()
            expandedEidsWithConfidence = set_expan.setExpan(
                seedEidsWithConfidence=seedEidsWithConfidence,
                negativeSeedEids=negativeSeedEids,
                eid2patterns=eid2patterns,
                pattern2eids=pattern2eids,
                eidAndPattern2strength=eidAndPattern2strength,
                eid2types=eid2types,
                type2eids=type2eids,
                eidAndType2strength=eidAndType2strength,
                eid2ename=eid2ename,
                FLAGS_VERBOSE=False,
                FLAGS_DEBUG=False
            )
            answers = []
            for ele in expandedEidsWithConfidence:
                # print("eid=", ele[0], "ename=", eid2ename[ele[0]])
                answers.append(eid2ename[ele[0]])
            print("output: {}\n".format(str(answers)))
            print('----------------------------------------\n')
            ap1.append(ap(answers, truth))
        end = time.time()
        print("[{} seeds]: MAP: {}, avg. time: {:.2f} secs\n".format(numseeds, sum(ap1)/len(ap1), (end-start)/20))
        ap2.append(sum(ap1)/len(ap1))


running days [2 seeds] iteration 1
input: ['sunday', 'thursday']

output: ['saturday', 'friday', 'monday', 'wednesday', 'tuesday', '24', '1', '4', '7', '9', '8', '21', '11', '13', 'christma', 'york', 'weekday', 'month', 'magazine', 'board', 'japan', 'earth', 'christma_eve', 'bbc', 'thanksgiving', 'north_america', 'birthday', 'year', 'campus', 'night', 'date', 'leap_year', 'paris', 'super_tuesday', 'ballot', 'communication', 'palm_sunday', 'halloween', 'game', 'york_city', 'german', 'wife', 'antique', 'young_india', 'news', 'season', 'mother', 'tokyo', 'state', 'easter_sunday', 'australia']

----------------------------------------

running days [2 seeds] iteration 2
input: ['tuesday', 'saturday']

output: ['sunday', 'friday', 'monday', 'thursday', 'wednesday', '1', '24', '4', '7', '9', '21', '8', '11', '13', 'christma', 'york', 'weekday', 'month', 'japan', 'magazine', 'game', 'earth', 'year', 'christma_eve', 'north_america', 'birthday', 'season', 'date', 'ballot', 'night', 'leap_year',

KeyboardInterrupt: 

In [9]:
len(eid2ename)

593252